### Refs
* https://towardsdatascience.com/how-to-use-torchtext-for-neural-machine-translation-plus-hack-to-make-it-5x-faster-77f3884d95
* https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec
* https://github.com/SamLynnEvans/Transformer

#### Necessary Library

pip install spacy torchtext pandas

In [ ]:
!spacy download en_core_web_sm  # https://spacy.io/models/en
!spacy download fr_core_news_sm

In [ ]:
import torchtext
torchtext.__version__

####  Download the [European Parliament Proceedings Parallel Corpus 1996–2011](https://www.statmt.org/europarl/) 

In [ ]:
!wget https://www.statmt.org/europarl/v7/fr-en.tgz

In [ ]:
!tar zxvf fr-en.tgz

#### Prepare datasets

In [ ]:
europarl_en = open('europarl-v7.fr-en.en', encoding='utf-8').read().split('\n')
europarl_fr = open('europarl-v7.fr-en.fr', encoding='utf-8').read().split('\n')

import pandas as pd
raw_data = {'English' : [line for line in europarl_en], 'French': [line for line in europarl_fr]}
df = pd.DataFrame(raw_data, columns=["English", "French"])
# remove very long sentences and sentences where translations are 
# not of roughly equal length
df['eng_len'] = df['English'].str.count(' ')
df['fr_len' ] = df['French' ].str.count(' ')
df = df.query('fr_len < 80 & eng_len < 80')
df = df.query('fr_len < eng_len * 1.5 & fr_len * 1.5 > eng_len')
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
# create train and validation set
train, vali = train_test_split(df, test_size=0.1)
train.to_csv("train.csv", index=False)
vali.to_csv("vali.csv", index=False)

### Flow of data
0.  Data
1.  Input Batch Generator
2.  Input Embedding
3.  Positional Encoding Tensor Generation
4.  Multi-Head Attention
5.  Residual Connection (Add) and Normalization
6.  Feed Forward
7.  Residual Connection (Add) and Normalization 
8.  Output Batch Generator
9.  Output Embedding
10. Positional Encoding Tensor Generation 
11. Masked Multi-Head Attention
12. Residual Connection (Add) and Normalization
13. Multi-Head Attention
14. Residual Connection (Add) and Normalization
15. Feed Forward
16. Residual Connection (Add) and Normalization 
17. Linear Connection
18. Loss Calculation
19. Training

0.  Data

In [1]:
import pandas as pd, pdb

df_train = pd.read_csv("train.csv", usecols=['English','French'])# , nrows=200) 
df_vali  = pd.read_csv( "vali.csv", usecols=['English','French'])# , nrows=100) 
display(df_train.head(8)) # df_train['fr_len'].max()

import spacy # a language-aware tokenizer library

spacy_en = spacy.load('en_core_web_sm') # https://spacy.io/models/en      initialize an english tokenizer 
spacy_fr = spacy.load('fr_core_news_sm')

def tokenizer_en(sentence):
    return [tok.text for tok in spacy_en.tokenizer(sentence)]
def tokenizer_fr(sentence):
    return [tok.text for tok in spacy_fr.tokenizer(sentence)]

from torchtext.vocab import vocab
import torch
torch.set_printoptions(threshold=100_000)

if 0: # Building vocabulary
    from collections import Counter
    coll_en = []
    coll_fr = [] 
    for sentence_en, sentence_fr in df_train.to_dict('split')['data'] + df_vali.to_dict('split')['data']:
        coll_en.extend(tokenizer_en(sentence_en))
        coll_fr.extend(tokenizer_fr(sentence_fr))
    
    vocab_en = vocab(Counter(coll_en), specials=('<UNK>', '<BOS>', '<EOS>', '<PAD>'))
    vocab_fr = vocab(Counter(coll_fr), specials=('<UNK>', '<BOS>', '<EOS>', '<PAD>'))
    vocab_en.set_default_index(0)
    vocab_fr.set_default_index(0)
    torch.save(vocab_en, 'vocab_en.pth')
    torch.save(vocab_fr, 'vocab_fr.pth')
    
    vocab_obj = torch.load('vocab_en.pth')
    print(vocab_obj.get_stoi())

,English,French
0,"That is not something we can prevent, and we s...",Nous ne pouvons l'empêcher et nous tenons à re...
1,"Thank you, Mr de Silguy.","Merci beaucoup, Monsieur de Silguy."
2,A mere trifle like leaving out the digits '19'...,Le seul fait d'avoir omis les chiffres «19» du...
3,"As parliamentarians, we should be reimbursed t...",Les députés doivent toucher des indemnités qui...
4,Since the EU was enlarged just over a year ago...,L’importance de notre puissant voisin de l’Est...
5,Today I would like to say here that if the Com...,"Aujourd'hui, j'aimerais indiquer que si la Com..."
6,"IDABC supports, initiates and manages the deli...","Le programme IDABC soutient, lance et gère la ..."
7,What I am describing here is the situation now...,Je vous décris là la situation actuelle et non...


In [2]:
vocab_en = torch.load('vocab_en.pth')
vocab_fr = torch.load('vocab_fr.pth') 

text_transform_en = lambda x: [vocab_en[token] for token in tokenizer_en(x)] 
text_transform_fr = lambda x: [vocab_fr['<BOS>']] + [vocab_fr[token] for token in tokenizer_fr(x)] + [vocab_fr['<EOS>']] 

if 0:
    # Print out the output of text_transform_en
    print("input to the text_transform_en:      ", "here is an example")
    print("output of the text_transform_en:     ", text_transform_en("here is an example aaa bbb."))
    print("original text:                       ", ' '.join([vocab_en.lookup_token(word_index) for word_index in text_transform_en("here is an example aaa bbb.")]))
    # Print out the output of text_transform_en
    print("output of the text_transform_fr:     ", text_transform_fr('Merci beaucoup, Monsieur de Silguy ccc ddd.'))
    print("input to the text_transform_fr:      ", 'Merci beaucoup, Monsieur de Silguy.')

1.  Input Batch Generator

In [3]:
import torch 
from torch.utils.data import IterableDataset, DataLoader

class train(IterableDataset):
    def __iter__(self):  
        return iter(df_train.to_dict('split')['data']) 
    
class vali(IterableDataset):
    def __iter__(self):  
        return iter(df_vali.to_dict('split')['data']) 

from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    coll_en, coll_fr = [], []
    for sentence_en, sentence_fr in batch: 
        coll_en.append(torch.tensor(text_transform_en(sentence_en))) 
        coll_fr.append(torch.tensor(text_transform_fr(sentence_fr)))   
    coll_en = pad_sequence(coll_en, padding_value=3) # 
    coll_fr = pad_sequence(coll_fr, padding_value=3) # 
    return coll_en, coll_fr
 
train_dl = DataLoader(train(), batch_size=8, collate_fn=collate_batch) 
vali_dl  = DataLoader( vali(), batch_size=8, collate_fn=collate_batch) 

input_en, output_fr = next(iter(train_dl))
if 0:
    print(input_en.shape)
    print(input_en)

2.  Input Embedding

In [4]:
from torch import nn

In [5]:
class Embedder(nn.Module):   
    # num_embeddings (int) – size of the dictionary of embeddings 
    # embedding_dim  (int) – the size of each embedding vector 
    def __init__(self, vocab_size, embedding_dim): # d_model = dimension of model
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        
    def forward(self, x):
        return self.embed(x) 

In [6]:
embedding_dim  = 512 # length of embedding vector
vocab_size_en  = len(vocab_en)
vocab_size_fr  = len(vocab_fr)

In [7]:
input_embedder = Embedder(vocab_size_en, embedding_dim)

input_after_embedding = input_embedder(input_en)

if 1:
    print(input_after_embedding.shape, input_after_embedding.shape, next(input_embedder.parameters()).shape)
    # prove each token has a uniq embedding vector
    # next(input_embedder.parameters()) is a mapping table between input_en and input_after_embedding
    print(torch.equal(next(input_embedder.parameters())[input_en[23,4]], input_after_embedding[23,4]) )

torch.Size([34, 8, 512]) torch.Size([34, 8, 512]) torch.Size([109946, 512])
True


3.  Positional Encoding Tensor Generation

In [8]:
import math
class PositionalEncoder(nn.Module):
    # max_seq_len means each sentence has at most 80 words
    # max_seq_len means 80 position to encode
    def __init__(self, embedding_dim, max_seq_len = 80): 
        super().__init__()
        self.embedding_dim = embedding_dim

        # create constant 'pe' matrix with values dependant on pos and i
        pe = torch.zeros(max_seq_len, embedding_dim, requires_grad=False)
        for pos in range(max_seq_len):
            for i in range(0, embedding_dim, 2):
                pe[pos, i    ] = math.sin(pos / (10000 ** ((2 *  i     )/embedding_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim))) 
                
        # we prepare a positional 80 x 1 x 512 encoding matrix
        # It will at most encode a 80-word sentence.
        # we choose 1 because the positional encoding is the same to every sentence.
        self.pe = pe.unsqueeze(1) 

    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.embedding_dim)
         
        # add constant to embedding
        # x.shape = [34, 8, 512]
        
        #     self.pe.            shape = [80, 1, 512]
        #     self.pe[:x.size(0)].shape = [34, 1, 512]
        # x + self.pe[:x.size(0)] = [34, 1, 512]
        x = x + self.pe[:x.size(0)]  
        
        return x

posEncoder = PositionalEncoder(embedding_dim)

In [9]:
input_after_posEncoding = posEncoder(input_after_embedding) 
input_after_embedding.shape, posEncoder.pe.shape, input_after_posEncoding.shape

(torch.Size([34, 8, 512]), torch.Size([80, 1, 512]), torch.Size([34, 8, 512]))

<img title="a title" alt="Alt text" src="https://miro.medium.com/max/380/1*2vyKzFlzIHfSmOU_lnQE4A.png">

4.  Multi-Head Attention 

<div> <img style="height:400px" src="...%0AIEltYWdlUmVhZHlxyWU8AAAAAElFTkSuQmCC" alt="Red dot" /> </div>

$$                                                                         
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V               
$$   

<div> <img style="height:250px" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHAAAADdCAYAAABjYvZ7AAAABGdBTUEAALGPC/xhBQAAACBjSFJN%0AAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAABmJLR0QA/wD/AP+gvaeTAAAA%0ACXBIWXMAAC4jAAAuIwF4pT92AAAAB3RJTUUH4gQCBDITphbDEQAAL85JREFUeNrtnXd8lNeZ77/n%0AfadJmtFoJIFAvQASTVSDMdgYXHHc4jhrpyeOHWezSXY3u3v35ubuTT7bsrm766w314nT7bjFHYMb%0AvYNAFBUkulAvCHVppGnvuX+cGSGJJkuCGRH9Pp9BvDNved7ze885z3Pep+hM4HIQwFcBC1AXbmEu%0ABy3cAkQwEoEvAmuYaKdxiQeBLuAAkBpuYS6HiSfr0rABDwN2YDawMtwCTeCTYQFQCcjg500UmRMY%0AB9CAfwAMLhDYBKwIt2CXE3YCg5EMLAPKgADQiiLwXsAUbuGGQg+3ABGIKcApoBilyJwB/gJFZB3g%0ACbeAE7gyQqPSpwEfUASkoOxCEW7hhiLihoQIgBH8O5AsgZoLIw4Tc+A4xwSB4xwTBI5zTBA4zjFB%0A4DjHBIGXhzHg/xGpgULkmBHJwFIgg8hYXJDAHMAPxAFPA+1Ehh3YgbJNSwBvuAUSqDXGHwFLgCgi%0Ao5EGyhdpvc8PNAC/Ap4Ld2PlAC8By+KcLtLTszGZTMhIa7IIQlvbeWpqKwkEAl3Av4Z7CL0XuGlS%0AYhLf+dYPWLxwBUKE+5mKXAgEbe3n+c0Lz7B1+0cOKeW3wkmgBkwHTPlzF3Pb8nuwWm3Iie53Rbhc%0Aidx71yPs278Dt7snLZwECoJKlDM2Dl03YRjGKE9540MaBg6HE4vZgpuesJsRsv+fMYAQAiG0q/w+%0AnCF68H5XOu5q1xy679g02oUWCzeBYwYhBEdK9vP22hfo7um8qLF8Pg8fbniLfQe2X/VcHk8vnV3t%0A6v/ePt7/8HX27Nty0fAuhEZDYy1vvP1b6uorr0pQcWkhtcPY75PghiEQoKS0kBdfeY5jJ0oG9QpN%0A06iqOcNLr/2cg4d29RMhhEDTtOC+qlF1XefEqVI2b30PIQRer4ePNr3D62//lo7OtiGNL9m9bxOv%0Avfkr6uqrLtsThRBIJKVlB6itOzvsHjsc3FAE6rqJjPRsSssO4vN7+783DIMjxftJS80kKioaUI3a%0A2+umtraS+oZqvD4PQgj8fj+dnW20tDbT2+fGMAK4XAlYLBZOnCrtJ1AIQXt7C2cqjjN92ixCJqPf%0A7xvUUw3DwOfzBeeJsX8nHG4zYkwhpWTWzPl0dLTS2FhDWmo2UkpaWs9RU3uWBfNuxu3uRghBZfUp%0ANm15D7fbjd/vIyFhEg8/8EVqas/y4YZ3ON/ShMVsY/Xt92Gz2pgxbSZHyw6xaP4tmExmhNA4drIE%0Ah8OJw+5AIunu6WLztnXcfusanLEuhBBUnD1J+fEi1tz9CNdijeKG6oFSGiS4JpGakklp2SFAzVNl%0Ax44Q53SRkpyOlAZen4ct29eTnZXLU1/7G5782vfo8/Ry6PAepuXMZPmy1cyetYC77niI6Gg7hmGQ%0Al5tPV3cn9Y3VaJqG1+ehpLSQubMXYbFGIQCf18OZimN4PH3Baws6OtuoqDyBcY3MoxuKQABN08mf%0AcxMnTx2lx92Nx9vH0fLDzJu7BLPZ0r/Kc/OSVdy06FYMaeD3+xAIzjU3Emt3kjQ5GVdcPMlT0zHp%0AZqSUuOISSUvJpPToIYTQqKurpKu7g1l585EyaP70z6kX5BGo764VbqghFMCQBpkZ0xFCo7LqFDZr%0AFG53D3m5+ZSUFqqb1s0IYOOWtQDYYxy0tbcQ63AhkRjSQEqJlAYyqLQLoZE/9yY+2vA23T0dFJce%0AICN9Gq64xLAuPtxwBCIlMdF2pk+fTXHpAUwmM9Oy83DGKnI0TaO+sZoPN7zFmrsfJT0th5hoOz3u%0AbrweX/AcF9t+0jDISJ+GbtI5XLSPisqT3L/mMYQmhlx+oJUm6O3rIRDwX7MV+htrCJXB/iIhf/Zi%0Ajp0ooahkP/PzlwICNX5KOjvb6evrIz01m1i7k9a2ZsqPFeH1eZBSommCHnc3bnfPoJ4YHRXDzNx5%0AvLvuZawWKxlpOUhDBveRWCxWvF4fDQ01CCFwu7soKSvE7/eHBGSsX27cUD3Q4YjDYrFgSIMpSSmk%0Ap2Th8/tITclEGgY2WzR2u5O0lCxccQn87qVniXdNwu/3kpOdR0NDDfUNNSRPzaCj833++NZvuOfO%0Ah0mIn9w/f86ZtZAt2z9gXv5SrFYbfr8fZ6wLqzWKmGg78/OX8P5Hb1BcehBNg+ioaGIdTiRgtzux%0AWaMYSxLDufSvA/8FfPvTD32B733nH0c92Xd3qxWYmBiH2u7pAimx22MB6OvrxefzEOtw0dXTwekz%0Ax/B4PGSkZ5OYkERr23liHXHYbFG0t7fQ29dLQvxketxd2GMcQRIN2jvaiIlxYDFbAOjsasdqsWKz%0ARePzeamsOsW55kamTElhyuQU+jy9uOIS6enpwmw2Y7VGjbzRNJ0jpQV8/38/TXtHW2T0wLF6ihwO%0AJ0C/UuEIEhfattmiiYqKRkqJw+5k0fxbQIBhqGEyaXJKv0bpciXiQiClgdWaOGD1RiPeNVhxcca6%0AkFJdx2QyM2P6HHJnzO0fWmNiHOqaDmdQCx5dDxzYXuEmUAAEjMCYnGyoNnixdij7zQgpJQEZGLK/%0AMeRYOazzDt2+3FuVMdFW+x84da5wEmgAnQDV1RV0dLYyKXHKxPvAK0KNCBVnT+LudQP4wv36+37g%0AJavVFnfXHQ9wy9LVmIPzygQuAQl1DVW8u+5lqqorALaHm0AH8M/ANwCb1WpDG8OV+hsRPr83ZJaU%0AAn8dbgIBEoAvo2LSk7lYp9E0TTMbhvGnEJdnCt6/7wr7uIFC4NfA/kggkKDQCUAsQwjUNC1T1/XV%0Afr//FSnljUyiBG5DjUofXmE/L9AM9IVb4OHiq8BhIDfcglxjmICfoxIqxIRbmLFCLLAW5cz63XAL%0Ac40xGxXa3QTcOtyDIl1jWAjcglq1eQiVPelGhADuATKBScADhN9GHzVMwDNcSPfRHryxGxGTge1c%0AWO0uAaaFW6jRIhc4NuCmJPA7VBalGw2PoNJ6he7TC/z5cA6M1CFUAKtRmlZD8KZKgRnAzHALN8aI%0AAe4CTqJWpjxAOSq9V8LVDo6EUK5LwQI4gbeAuaggmP8AXkTZSA3hFnAMEQf0AOtR86AJ+B5wEDVt%0AdIZbwJEg5H9nBzageuB3gr9F6qgxmnsFNWWcRZG2cshvl0WkajqhFe1LkXWjBVCE7nVoXpqBv10W%0AN9rT/CeHSCcwwIWncOI90yVwuSFUAEmoHGHhfL8TDTiEED4pZSZwM+FzA5EoVb86+DcicCkC7ahc%0A0V9GJR0IdwyhQ0qpAU8BXwqjLKDeBBwCngN2EAHz8VByzCgD8h+EEI4YewxmszncMoYQG/yEDb3u%0AXvr6+jJRGSyeAnaFu1GGEpgHfEPTNceqe1fz8BcewW6PmZh8UENBdUU1Lz3/IqePn85FvYQ+hOqV%0AYcNQAmcDKckpyTz5199g5vyZSGOCvhDm37yAnu4e/vNH/07AH1iIWsOsDKdMQwmMAnRXYjyJSYkE%0AAoEJAgdAN+lMmjoZXdcJ+APRqPYKK4YSONCX7hOfLBRPEIopMAzjE3mZhY4d6pZ3ue8H/q5EvsYP%0AmxxZu1xLjJmGKYSgr7ePylNnOdd4jhh7DBk5GSRMHuYrPAFdnV10tnUyaUoiFqu1nxB3t5vO9g7i%0AJyVgtlysVHW0tuP1+UhITLgo2ORGx5gY8kII2lraePX5V9jw7gYqTlRwcHchz//kFxRsLxjW8a1N%0ALbzw7O9Y99pa6qrq2bt1L73uXnRd50jBYf7HU39H8YEiNH2wyJ4+D7/4yc/5xY+fw+vx/MklChqb%0AHihgz+bd9PX18eT3niLGEUPAH+DAzv2se3UtGdnppGSmIqVUAf+hv0ikIdF1nYqTFQQCBk9+7wm6%0AOrrYu3k3s+bPQnNquHt66Wjp4MDOAyxctqifRE3TOHvyLCePnmRKytT+IVZoAoEYdD0pJZqu9UcT%0ADdxvPOenGRMCpSFprG0kPTsdZ7yTgD+AbtJZvOIm2lra8QcCIKCzrZNDew5SW1VL/KQEFt68gKlp%0AydRV1XFg134a6ho4sv8IredbqK2qZd/WPdy86hYEMGvebNpa2qivqSc9Ox3DMDAMg8P7DjEzfxbS%0AMPqnp5qKGo4UHKa9tY3ktBSW3LoUq83KqfKTZEzLxOFUwS8N1fX0dPWQnZszbofeMRtC03My2LNl%0AD7s27qKxrhGvx4vFauG+z95HWlYaHa0dvPKLlzlx9ASpmal0d3Tx+//+PZWnKunu6qa58RztrW1U%0AV1RRV1VHZ0cnNZW1uLt7kEBiUiJpWWmUFBb3X/N803mqz1Sx4OaFIFSPqqmo4Y+/ehUjYJCencHx%0A0uO8+/I7BIwAhbsLWffqewQCAVqbW3nl+Zdpqmsat+TBGCoxt951KwG/ny3rN/H+6704XXFMmzWN%0AZbffQnJGMoW7C/H0eXj677+J3WEn4A/wzktvs3ndJp78m6dYuWYV5UfKefzJz9FU30RnWyePfvWz%0ATEqaRNnhMjRdY8HSBWxat4nb16zC4XRw9FAp8ZMSSMlM4eCeQgSCxtoG5iyay/2PPYCma2TkZPLi%0A/3sBn9fHA48/yG+f+TXbP9hGbWUNSclTWLJyabg5GBXGZgiVkih7FGsevY+V997OuYYmKk9XUrS/%0AiMN7D/O1v3yC08dOMWfRHGIcMfj9fjRNY+HNC/njb/5IT1cP/WvUcsgndA1Dkp2bA2yi4kQFefl5%0AFBcWc8sdy7EENVNDGsy9KZ+muib27yigt6eXqjNVtLe24ff7mZw8mYc+/zDP/PA/ycjJ4Ns/+C5m%0Ai3lcB9SMeggVQuDucbNl3WY62jqIccSQNSOb1Z+6g7/4X98mJSOFPZt34/f5LzIBTGbzRfadvMzC%0AnZSSGEcMefl5lBQWU3Gigr7ePmbmz8QYsNhQfKCI999YT3trO854J3n5M3E4Y/vtN8OQQdIuf63x%0AhDGZAw3DYMfHOzhWVN6vIUokFquF6JhodJNOamYap8tPE/AH0HQNoQlOHzvVv89QAzmUkGBQogEk%0AcxfnU322mo3vbiB3Ti7OeGd/D/J7/ezevJtFtyxizaP3sWj5YuyOGPp6lTnSVNfEuj++x+NPfo7U%0AjFTefekdvB7vuDY9Rk2glBK7w87yO1fw4VsfsGX9Zs4cP83p8lOsf20dp8tPcdOKJdy8cilN9U28%0A//p6Kk5UsGfTHnZs2MGtd9+GxWbtV/UBLFYzvT1utn+8jeamZhhgciSnJ2N3xHC44BALly0KNv4F%0AM8ERa6f4QDEnSk9QuKuQnRt24u52c6r8NGtffpes6Vksv3MFD37+IZrqGtnx8fZxvVw41CttPnB/%0A0tQk0z0P30uUPXrY78EzcjKIdcZyrOQYRYVFnDx6Ak3TWPPop5gxewZ2p4O07HSOFR/j8N6DtLW0%0Aseq+1Sy4eYFKjCMlroR4UtJTsNis2GPtNNQ2kJw2lbj4OFwJ8aRlp6GbdOJcLtJz0pm7OL9fg4yK%0AiSZzeiZpWelUnjrL4YJDtLe0c/Pty8iekU1bSxvxk+JZ/anV2KJsRNujSU5PoaOtg5SMFHTT1R30%0ANE2j4kQFWz/cQiAQaAdeAc6PERehmr1RwB8Z5iL50LHjq8DzcxfmW5954afEJyV8oqdT0zT8fj/e%0APi8IsNqs6Lo+yMAO+AN4+zyYLGaVUWLA/BcyukPnGtgrB/52qbXPgQsEgUAAb58Xk8XUP+9KQ150%0AXGgxYbgPqa7rbF63iR/8xffxeryVwH0o5+OxQB7wEeBChdptH85BQ7XQC+n0RjAvGIaBpmlExUT1%0AN9QgBcWQwd+jL/ptKCFX+u1SWmPoO5XnRckgpRz0AF49hn44iKz5ciiBPsDo6erG3e1m0pRJBBh5%0AAoIrKQfXQ3EY62sITdDT3R3Sen1cORDzumAogaeA1trq2uS3//AWn/7SI8TYYyb8wQCQ1FbV8v4b%0A6/H7fKBc4ZvDLdXQRzQa+Dfgz80Wsyk5NQVHrCPcMkYEDGlwvqmZ5qZmpJTNqLKsb47hJcZkDnSj%0ACDR8Xt8jVRWViYR/0JcRIEMIfuA08N/AunALA5deSqsHvg+8jAo4DKdbmhRCuEwm8wyfz3uQ8Lvx%0A9aCqW5+NAFnGDe4E8RYqcvVGRh4XgltuH+5Bke5abwY+DfIePkHc+J8SIp3A6ajgRzuqLHjYvcAi%0ADZFMYCjwPyu4vRLltzqBAYhkAhNRmSlCilYqKrdaJMt83RHJjbECFbXaizIlOlE9MjXcgkUSIpVA%0AC8pt/Z9RWZoAfoNK+pMdbuEiCZEaYh0AXkXNgw8E/1YCz3NjphkZMSK1Bwa4EEQpBvz1A93hFi6S%0AEKkETmCYiHQCJ2Lkr4LQHKijNL58whwFOwQ2IC34/xWoLEaRsrAtUQmHDhHGxEMmVKqnJ4BvEf6Y%0A+MvJCPAoKqdYJKEPOAD8CxeS1V1XmICvAD/SNC0+fWoKyZOnIK5hta1RQCeCUoN5vV7O1FSaW9vb%0A7kClBfsScPx6y2ECviOEiL97+e386Jt/Q/qU5HC3zbhAwDDYcaiA7z/7r9Q01C0EHgROcJ17oQmY%0AYTFbePyeB1kyax6BgH9CWxgGhBB8etU9bCzYwR/WvqGhXgeZuM5+MiZA0zUNR1SM8hQbx3EC1xNS%0ASnRNJ94ZF/rKTBi0+v4LXi1OQLtKDXZNaKPyAht+jfexud5wzj8shPl5H5aUhpTUnz9Hc2vLJX/3%0A+f1UN9bR1TPyRZKO7i663MM/vqK+mraujlGTOPR4AfR5PZysOUsgMDY1na4lrkqgEII+Tx8/fP4Z%0A/v5nP6bL3TPopjVNo/BYCV/4wXfYfrgAXf/kiqImND7cu42thwrQr6IBCyFoamth7c5NeH0jn240%0AoXGiqoJdRYWX/H3j/l2UV56+pvVvxwLDki4gJe3dnZScPk5ZxalBw4vP72fLgd20dnfi7usLunYL%0A9GBkkSa0QY0Q2hbiwj4I6Ojpptt9IU4wdOxQCCHYU3yQhNg4kuIT+z2xNSHQguccdK0B1wvtB8pN%0A/nRtJXtKDg166CQQY4tmZuY0thbuxecPu+/uFTE8o11K7DF25ufNYW/JQZbOntffwGcba6hpauCW%0A+YuDbS/o9faxt+QQR08dR9d1lsxZwKK8Oei6ztmGGnYfOUBzawvZ6ZmsWrSMOLvjgk+/UL267lwj%0Ax6oruGnmXGKj7P1zdHevm6ITZTx6531oQsMb8HKgrJjCsiJAsHDWXG6aOReTbuJ0XSUmzcSJ6grK%0AT5/A5Yzj9sXLyJySysnqCvaXHqG6sY4t+3dz89yFRFmtSIKBotNy+WjvNmrPNZI1NTVilbthjw8C%0AuCV/EZUNtdSfb1K9Q8Ce4oNkJaeRlpSMlAaGNHht4zo2FOwkfUoKsXYHv3r3FQ6fLKOxpZn/fPnX%0AdPb0kJWazv6jR/j9+jfwBfz9w3JoaPvZGy/S1tGGzWLtJ08TGvXNTXT2dJM2eSqGNFi7YyMvvP8m%0ArlgniS4XL3/4Dm9s/gApJe/t2MS/vvAcxytPk5WSTntXJ//+h+c5U1dNW1cndecaaW5rpbK+Bn/A%0A3x8PIqUkPjaOOHssJ6oqEBFckGvYy2aGlGRMTaGhuYn9ZcVkTEmhrauTQ8dK+cwd97Gn5CAgcPf1%0A0d7dxdOPfIG8rGkYgQCNrecpPl6GJjQ6u7t4/J4HSEpMYunchazdvgF3X7AMkBAcKCvilY/f5c4l%0At3L30lvRdX1QNFF9cyMWsxlHtJ3a5kY+3L2Nv/izr7Bk9jwA5uTk8cwrv+aW/EV09nTjcsTyzUe+%0ASJTVhj/g56ev/Zb3dmzkb7/0DepX38vRs6f46kOPqQLKA3qZWTcxJXEydc2NER3JO/xHS4LFZGZZ%0A/kIKy4ro9XgoPXMCCeRPz+uvKhlts/H5ex6krbODdds38vrmDyg5WY67t5ec1AzSpiTz01d/y6sf%0AvkNDcxOP3/0AsTF2pITdRYX824s/p6e3l9sWLh1EnmIQ2ru7MGs6FpOZ09WVOKJjmJMzg4Dfj9/v%0AJy8zh8nxiZyqrSLKZmPpnAVEWa34A350XWfp7Pmcqa2iz9M3IBb/EgQJsJhMtLW3hd1UGBsCUUPL%0AgtzZdPe6Ka88zZ6iQhbPyifOHtsfm9fZ082L77/FrqIDCCGYnprBzKzpCCFwxtj53hee4r7lq+j1%0A9PHWlg/5xVsv0dnTTcAI0NbRzncffwKL2cza7RsuKUNIGUGAz+9D13U0oV3Iki4EFrMZf3BYHqoV%0Am3QdIxC45JwmEP1KDoDfCGAymSLn/cdoCTSkJNHpYlbWdF7fuJ6K+hqW5y8e1HjHzp6mqr6Wpx7+%0AHA/ceifzps+kz+tBCNhVVMimgp3cOn8JTzz4GH/35aepPdfIiaoKLCYz961YzapFy/j6Q4+zsWAn%0AB4+VDjYrJMTHxuGXBj6/j4ypqbR1dVB/vgmTbsKk6zS1nqe2qZ7USVPo7e3lyPGj+A2jX+M9UX2W%0ApMRJg+bW0G89fW7qz5/rT9LX2d1NQlx8JPM3fAIHKhIr5t/EtsI9JMUnkjE1WWVJCsapO+0O2ro6%0A2XlkP8Uny/njpvc5VVNJTVMDmqaxcf8u1u7YQNmZE+w8fACf30+8M64/qUHACDBv+kweuPVOfrfu%0AderPnxsQWWuQOnkKHq+Hts5OslPSyZ+Wx/NvvcyBsiIOHTvKc2/+genp2eRmZGMymSg7e4q3tnxA%0AWcUp3t+1he0H93HPspXomo49OoZjFSfZWLATr89HYXkx/+eXz9DV68br89Hc1kLG1JRwc3RF6MCP%0AzCYzn75jDTOzpmPIi2M2BAKTrpOXkYPT4SA2xoHLEcuqxbcwNXGyGj4RpCelkJuZQ7Qtih0H91F8%0Aspx4ZxyfWb0Gn9/Hwrw5ZKeks+NQAQUlh+lwd/PoHfcxJ3sGUkJy4mSSXAkgISslnUAgQJzDQYLT%0AFXyIlI1WfOoYcY5YspJTmZk1nbauDrbu383RihPkZubwxTUPY4+KoaDsCPNzZ+PudbO5YBeNrc18%0A5o413DxnISBxxTrx+9XQPTNrOgSH3/xpeTS3t7K/rIj7l99BdNTlHcI3H9hNQfEhUAWr1sGII2JH%0AHCMvo21R/OFfnuXTq9YodfpSOwqhNLXgtpp3BsewE8y9IoSgz+vBMAyignVxQ/tqQsPj8+L1ebFa%0ArFjMZgzDCJnv/T1dBM85dK7SNI2tB/dysqaSJx/8M3RNzXG9fX1IINqmnNaklPzkpeeZP2M2a5at%0ApNvdg9lkxjogLj+0/ioNeeG6we/W79rC+fZWvvKpz1y28STwP3/2Y/7rD78CFc31JMprYCQYUXzg%0A8IdQOViZNqRxUdx6aA8pJTazlWhb1EX7GtLAbDJhj45RCkWwMZVCOOB8cElFQxqSxTPzMek6jS3N%0A6gmUkiibjWibbVBiBJfDSZTVikAQExWN2WQaHLMfjNMfaiZ097o519bCqkXLBik1lyYxvCqqCfD4%0A/H5rVWM9EolJ1yNZawYgzu7gsTvvx2Iyq2Wyy+z3ubsfxGo2I8Tw3y4IIMpq47N3rMEZ4wAhLukG%0AoAlBS2cHZ2uqQ1/1MPLhc8QwAbt9ft8dv3rrZcwmE7NzZqBpOhFt/KDm5WB/H9U+lz/ySr1L4O7r%0AZeO+HWw7sAeU+/8+lN/qdW4HVafvWWCOxWzBHh0dqT4xEQWfz0e3uwfDMAIopeOvGF3SnxHHyG9H%0AFfv4c6/Pu7i1wxtNZHU/QRhcFYaJc6hG/yVjl7HpE8GEivXeDRwBpqIyVUQMhBBpuq6v8Pv9b6NK%0Ak0YKDKANVXX6ug+dIQxczO5BZWCIKEgp8/1+/2pUdNKZcMsTaYj0yc6BCq1eiIoNnMAQRDqB84Hl%0AqJHiISA+3AJFGiKZQBMqNnBycPsmYHwnuL4GiGQCM1DpHEN2ugs1nIazIGXEIVIJFMAdwb/nUGbN%0AcVTdvrxwCxdJiFQCzaiQrW+iVvkF8ALwQ8AabuEiCZEWShaCD/gAZZOGVp/dwCYiKEIpEhCpBIa8%0AVS41QkS+u/R1RKQOoSFE0pJeRGKseqBpDM81EDbUQyZR8+K1SDEiUUt04/JhGW2j21Ha4h0ol4Cx%0A9P8JkTZTCGFIKT8HLB7ja4BaxzyFmnOP8CeUB9QJ/F+glYsrHo23j4GKrn2U8E0rI8oXOtIeKFAJ%0AB75ltVpili+fR/7caZj0SJ9SL0ZXdy87dhwWJ05WzZBS/hAoD37GBUZKYDRqmStmzb238LNn/5Ip%0AU1wR7T95ORhSsnNXKV9/6sdUVTXMRKW1HDcEjrTL2IBJuq5x64p5pKYlogkVGzLePiZdY+mSPObO%0AmQbKxhxXWR5G2gODc4fAbNKHXZpbCKFqHYVcFA05wuopg88JoylBrhyKbVH9CzzjaiAZteo/XLc6%0AIQRt7d2UlVfR0tpOYoKLvNw0EuIdoyKxq8tNIGDgdMaM6i7GK67LSowQgpOn6vjVbz7AYjbhdNpo%0AaS3npc4enn7qARbMzxkRiZqusXV7CS2tXTzxlbvGdSXOkeK6EOj3B3j1ta1kZU7l60/ci81qprfX%0Ayx9e3sRzP1/Lf/7HN3HGqh4kNDGg0pjsr86paaGhcnAls67uXtrbe/q/03RtwJA+vkuMDwfXnEAh%0AwOPxUd/QyupVC4iyR4HfT3S0lUc/cxu6ruP1qlCw9vZutu8s5vSpGhIS4rjttnlkZ01BSklZeTW7%0A95TQ0dFNTnYqq1cvwBXvQCD6C635fH4Kdh/nSNFJzGYzK5bPZc7szJEUYhs3uOaGm5RgtVnIykzi%0ApVc2smtHMQ2NbXi8PhITY/n61+4hMSGWjo4efvrsOxTsP0ZmVjJd3b38+CevUnG2kaNlVfzsubWY%0ATCayc1LYvfcoL7y4kUBA9qschpS88dYuXn9zO0lJ8URFWfnFL9ezr6B8XJcZvxquyxBq0jW+/rV7%0Aee31rfzq1+vx+wIkTYnnpkW53HXnIiZPcbFjZyktLZ380z9+DVeCA7/Xz29++xGnzjTg9XhZdfsC%0AHvvcKkCQnprEL365DndPLwI1vNbXt7B7dyl/+d1HmDU3EyQkJyey/oMCFi2cjtUazgpC17Btr8dF%0ApJQkJbn47rc/TVtbN9U15zhaVsm2HUXs2nOUH/7Dlyk5eobFi2fgirdj+PxoAr765bswpMTvC1Bd%0A08ymDYV0d/dRWnqW9vZONb8JEEKjsrKRMxU1vLt2B+vf34UQgta2bioq6unodJM0Oe6GVHKuwxwo%0AqK07z4HC49x37xISEhwkJMayYMF07luzlO/97c/Zu68cvz+A2TxYHJ/Pj88fYMfOEgoLTzB//jSS%0AUxJZuHAGp89UDzJhvF4/sY4YcnPTMZlUbIchYeXKBdjtUYxnU+FKuC5KjNvt4eVXN5OXm87suVkQ%0AMEBIYqKtREfZ0DTBjOlplJVV4vP4MFuUWG++vQsQHCgs54ufv5MVK+cBgm2bD9HX51Gx8lL18OTk%0ABJxxsSy/JZ+pqapO1sljVWzdXsTKFWK4aw3jDtecQMOQZGYkseSmmTz7s3d49JHbSE1NoLfXx9Zt%0ARXi8PpYszgUBO3YW85vffcyKFXM4c6aegv3l/PnTD3Ls2Fl27iolzmWnqamdDRsO0tHhprKyKXgN%0Ag6ysKaSnT+K/nn2Lxx5bhd8f4LXXtpGfn43NdmPOfzBy/5Io4HFN0zLuuXspS5fOvOL8YjLp5Odn%0AI6Vkf+Fx9hUcpby8ioQEJ0989V4yMibjcESRl5tOUfFpduw8QktLF499dhWLFk0nIz2J4tIz7NlT%0AQnu7m/s/dTNZmVMxmzQy0pOYPCmOzMzJzJ6VSUNDK9u2H6a8vJLFi3N55OHlWCyXf06FAL/fYO17%0AeygvrwDYBWwNAxcjCrG+bkpMrCOaP3t0JQ89eAsejw9d14iKsqAJ0W+sT5+ezN/89Wdx93mwms1Y%0ArSYMQ5KXm8oP/ufn6fV4sVnMWCwmliyegSFl0MZTxv+kRCdPP/Up3L0ehBBEB9c3b0TlJYRREzhc%0ACysU+mwx6ViCysrQBLOGIdE0gSMmCuSFVRjDkOi6hiNYWtwYVFocQgpKiKiYaNug7bG7i8jDqAlU%0AxvTwG0C195Ub9nINP1xCPlmPE0jDwOPpj1wbV911pAR6gdZAIEDBgTLONd3BpEmx4/I5NiQUFVdQ%0AVnYGlMtiU7hl+iQYKYE9wMfAPevW77K53R7m5Weh6+OPwp4eL5u3HuLs2XpQ8ZE7wy3T9UIC8Byq%0ASFW4nZLG4lOJqqERLs/v6+rUBNAC/ACVnWE1itBrAbOmaU7DMK5VDLqBarh3g/cyrjy/R6vEtKMy%0AFL3BNao3L4TIBz4jhPh3KeW1Kj3XxzgjLoSxsgO9XKMEBFLKFVLKR1EPyYHr1TDjBZHuyJmEcl/M%0ACP6diEwagkgncAUqTh5UtO5EAeQhiGQCo1AZixzB7VnAqnALFWmIZALnoggLrYxYUe78znALFkmI%0AVAJ1VHqR/Vwo6fYxKpJobriFiyREKoEaKpz6r4Eq1Grzx6iEcvXhFi6SEKkh1j7gKGr+C63PSaB6%0AxGe8QRGpPTCE8be4ep0R6QRO4CqIdAJvbL/4McCV5kAzKgbeTHiGMomaA0OptRyovGnhGlYN1Jpp%0ANxH00vdSBOrAMuBx1CuOaMJHoA7kBrefAu4PkyxwIcHrXuBVhul0dK0xlECBSuv4f4GcS9UeCiMy%0Ag5+wQEoIqJoa9wK3Ad9BZbcIK4YSmAH8vRAiZ1rOTFbfvgZnrCtyxoswIVSb4mzVabZsfV9va2+5%0AG/gG8L8Icy7voQTOB2bFxyfy7W9+nyWLbwuJH04ZIwICgdfnwWF38OLLPxeGYaxGzcl14ZRrKIEu%0AwJI0OZmcbJXVUWXVnwCAxWJj+rTZmEwmvF5vPBcW2sOGoWaEBIL1+cQN7RA7MlyI/B36n3DhUnbg%0AyIqdDECogNQn/e1S+16P845njKkhL4TA7e7m3XUvc+jI3ksSUFV9mjfe+R3N5xuv2JBSSjo62/D6%0AlJt8R0crb7z9e8qOHbnkeU+cLOWtd1+ko7PtiiR3drWzv3AHPl8klaAYOcZ8JabH3c26D17j7fde%0ApMfdNagxDcNgy/b1vPG2IjBUkCpUVnXwvgE++PgNamoq0HUT7R2tvL32Rd7/6HW8Qxrf5/fywYY3%0AeXfdS1cnsLON/Qe3qwcj/CPgqDHmBEopSUxMIhDwU3H2RH8JbyEE55obqKuvJic7l2CeIKSUtLY2%0AU1V9hrb2C56DfX29nG85R2dXBz6fV8UATk2js6ud2rqzap5Gzdc1tWdp72ghI0OlKwkEAiGbrR9+%0Avy8Y0RuqWD/+yYNr8jpJEhNtJzNzGiVHDzJ75sL++elo+SGSklJwuFXRZJ/fx7YdH1B+vBgjYBAw%0A/Cy9aSUL5y/j/Y/f4EjRfpqbm/B6PCQlJeOMc5GWkknJ0YNkZ6oFGiklxaUHyM7K5XxLEwI4Wn6I%0Azq4OViy7sz+oZuOW98idPoeoqIiqLDRqXJPFbAnMmbWIc+fq1VCpabjd3ZQdO8L8/CWYTGaEEJyt%0APMnR8sM88uCX+OaT/4N77nyY3Xs34XZ3c9vyu8nLncOqlWuYPXuhqgyKxrz8JVRWnqSzsw1N0+jo%0AbONMxXHm5y9FCDUMN59vpL4h9B5YSXS28iRtHedvKAUGrhWB0mDypKkkJEym/HgRmqZztuok0pBM%0Ay57ZXyU6zuni/jWPMSlxCh5vH7puoqurk95eN1OSUolzxjN1SiquuIRgKLVBWkoWVlsUJ0+Xo2s6%0Ax0+WEBPtID01GzmwtOqQEnpC026IOW8ortkbeZPJzNzZiyk4sI1lS1dRVHKAGdPn4LA7++3L6Cg7%0ARSUHOFy0F6vFhiEN/H61MhUq2yoHlnqVYLVGMTtvPkfLD5E/ZzHFpYXMnbMImzUq7OVQw4Fr9j5Q%0ASoNpOTPp8/RxpLiAhsYa5s1dPGifbTs/pKGxhlW3fYp77nqElSvW4LDHDllAEIP+K6VkZt58WlrP%0Acbh4H11d7czKm39R9W1pGP3Ru4FAgN7ennC39TXB2GuhXIjGdca6yM7K5c13fk9C/GSmJKUFc5ep%0A4sONTXXEuyaRNDkZs8nMiZMlNJ2rJ2AEEAgChp+uoBYKBIdeg8SEJKYkpfLG278lIz2HeNek/mtK%0AICbaQX19De7eHoSAyupT1NRWBDXiULHmG6O3jvkQatJNJLgmYdJNgGDenJvYs28LC+bdjMlkJhDw%0A44yNx2aLVtrmR69zrrkBIQQ2WxTTcmZScvQgaalZZGXMYOOW9zAMg9SUTOLjJ6FpOrquMz9/KcWl%0AB1mQfzNCaGiahsuZgK7rzJg+h70FW/ntCz/F5UoEDHJnzMFqtaLrZuKc8cHS6OMfQ2f1rwLPz5m9%0AwPpv//Rr4l2Jn3g9NBDw09nZjiM2DpNuwjACtHe0EetwYjKZkVLS2dVOVFQ0ZpOFuvpKamoriXU4%0AycrKxQgE6PP0khA/Gb/fx/mWc0RHxRATY6erqwOn04Wm6fj9fjq72nE6XeiaKmfe2dWOPcaB2Wyh%0Avb2FUxXHQEJOVi5CE1gtNiwWK909XTgczmFXtg5B13W27viAH/3zX+H1eipR7v7HxoiLEdfQHUqo%0AmjlGqLDpuon4+En9xGuaTsKAbSEEcc74/u30tBwy0qchg1omgMOhFB2z2Ury1HRADY8Dz2symQad%0AV9M0XHEJ/UNpXFwCSxffBnBR7fiB1x/vGEqgAUifz0sg4EcIDSkDfNJVC9U4Yljbg/N5Dnz3GMoP%0AavRvf5LzhlZkLsbF5xnmXSGEUoiC5BtEgNPVUAIbgN66+mprwYEd3Ln6weBcNgGAtvbz7C/cETJ1%0A6lE1M8KKoewcBnZ2d3c9+Pyv/529BduIjXUOO6n5jQuBRFJXV8XR8iKklB7gHVSYecRhMbAZ5UIX%0A7sQDkfhpAf6Lsc8JMGZJDg4CXwPuCp5U5+pGkzSZTGlSSlsgEDhF5C71C13XZ0sp6w3D+KTDn0Cl%0AV9kP7ED5h4Ydl5vgaoDfMfz3LsLvD/wtyGTgH7hG8fJjgLhAIPALYAPw5giOD7vSMhRX01CGu2Qx%0ABeQDqJj25xk722issQC4ExVnuA7oDbdAo8VYLaUtB+ahHG/vJTKHUDMXatHfhiqoPO4xFgRGo1YO%0A7Fyo/Z4Y7hu7BHKAu4P/T0a56Ud6cM9VMRY3MIfBWtMi4JZw39glcC+KxNB9388NkPVitASagg1h%0ARykuBhfiK6LCfXMDMBn4FBfmvD5UGMEd4RZstBgtgS5Uo/wAtTLhBv4VqCCyyrhlANuA/4dSysqB%0A/4PyrI6kB+26w4KaA6ejInW6UInvbERWw8SglJivoAjcjTLEHUROnoDrnq0QLuRIG6h1aqghKpIQ%0Aeh0fGnFC9m1XuAUbLca9FvanjgkCxzkmCBznmCBwnGOCwHGOCQLHOcaKQIF6b6gRmQvZA+Ucy/sO%0AO8bKiO0VQhwCEqSUHeG+qSvgnBCiACiVUvpHfbaxRagT6IShE+gWizXBbLZM5kJmpUiEzWq1JZlM%0AZheR1wtTgd8ArxGOnKi6rvcHXUYyzGZzJCUvGggNtSwZwydI7j5WXVWg1j5Dy2iRNjwNR/5x6Xo3%0A2kdRB25GZS36IvAg6l2gFeVXE9YsRkHkotwoznDxg5URlNuP8om9ntBQ7yizUIvYl8JtqORLp7nM%0AAzaaMc8MPAH8GNX1P0aVr2kBvotyboq7zo1yKeShEvdZL/H9j4FphCeNs4F65fYtLt1OMaiOkcU1%0Acqa6C/Va5mEu1mYXoDy//orwF+t4EOWEOzDb/YLgd/+EeqcZLuSgfHBvv8Rvi4K/XVGhGWkPjAI+%0Ah6o1u56Lh6YjwC9Rb+tTwthAIQwcflYA/wwUAD9BpZAMF6qBMlRnGPigC5S3QCVXyYg4UgKTgWxg%0AI5cvGrUX1fVnh7GBhuIeVK/7EPhvwu+c60O14UJgyoDv41G6RchD/rIYKYEqavLKE38HcJ7IcBwy%0AAZ9BuX4koWpRRMpL50IUkUsHfDcf5dWwdzg3dqPDQPmsOoD/AGYCfwfUAsfDLRzqId+H0pTfR41o%0Ad6KmoaumshxpDzwfPHbqFfaJQ3mDhbtQh0CNBv+C8sZ+HqW2/29gUphlA/WAbUYpNJmoEWJO8Lur%0A5vocKYENwUa4m8GTbyJq8nWg7EGBmqTDCYGyAQuD2x0o8yEWVRkmEpyvjqGKL69AzX09qB54VYxU%0Axfeh3Am/BjQHG8hA+Yd+HaW63wu8h3qSwrnKkYsaQtcBnuB3HUGZnwxuF4VZRi/KnFmD6okFwK7h%0AHDgaG60G9XQ/jdI0E1EKSxbKNhTA71G1j8LZOHkou28tFwgENbR3oQzpGpQvazjRiXr4k4FngHPD%0AOWg0BAZQT+4x1Ni9GFXjrxtV3foISm0vJ7y2Fqg5u4yL7dVTqN6oo5arwhk+1h2U5QCq913XXNcC%0AtZwWCnABNb9mBb+bwAQmMIEJTCDS8P8BFe6B4KW0BEEAAAAldEVYdGRhdGU6Y3JlYXRlADIwMTgt%0AMDQtMDJUMDA6MDk6MDgtMDQ6MDA7M0CpAAAAJXRFWHRkYXRlOm1vZGlmeQAyMDE4LTA0LTAyVDAw%0AOjA5OjA4LTA0OjAwSm74FQAAABl0RVh0U29mdHdhcmUAQWRvYmUgSW1hZ2VSZWFkeXHJZTwAAAAA%0ASUVORK5CYII=" alt="Red dot" /> </div>

In [10]:
h = 8
N = 6 

class MultiHeadAttention(nn.Module):
    def __init__(self, h, embedding_dim, dropout_rate = 0.1):
        super().__init__()
        
        self.h             = h
        self.d_k           = embedding_dim // h
        self.embedding_dim = embedding_dim
        
        self.Q_linear = nn.Linear (embedding_dim, embedding_dim)
        self.V_linear = nn.Linear (embedding_dim, embedding_dim)
        self.K_linear = nn.Linear (embedding_dim, embedding_dim)
        self.dropout  = nn.Dropout(dropout_rate)
        self.out      = nn.Linear (embedding_dim, embedding_dim)
    
    def forward(self, Q, K, V, mask=None): 
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        # (batch_size, embedding_dim) -> (batch_size, h, -1, embedding_dim // h)
        K = self.K_linear(K).reshape(batch_size, -1, self.h, self.d_k).transpose(1,2)
        Q = self.Q_linear(Q).reshape(batch_size, -1, self.h, self.d_k).transpose(1,2)
        V = self.V_linear(V).reshape(batch_size, -1, self.h, self.d_k).transpose(1,2) 
        
        # calculate attention score
        scores = torch.matmul(Q, K.transpose(-2, -1)) /  math.sqrt(self.d_k)
        
        if mask is not None:
            mask   = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
            
        pdb.set_trace()      
        scores = nn.functional.softmax(scores, dim=-1) 
        scores = self.dropout(scores)
        scores = torch.matmul(scores, V)  
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).reshape(batch_size, -1, self.embedding_dim)  
        output = self.out(concat)
    
        return output

In [12]:
mha = MultiHeadAttention(h, embedding_dim) 

In [13]:
_ = mha(input_after_embedding,input_after_embedding,input_after_embedding)

> /tmp/ipykernel_406703/2144835108.py(35)forward()
     33 
     34         pdb.set_trace()
---> 35         scores = nn.functional.softmax(scores, dim=-1)
     36         scores = self.dropout(scores)
     37         scores = torch.matmul(scores, V)

ipdb> Q.shape
torch.Size([34, 8, 8, 64])
ipdb> K.shape
torch.Size([34, 8, 8, 64])
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


5.  Residual Connection (Add) and Normalization
6.  Feed Forward
7.  Residual Connection (Add) and Normalization 

In [20]:
torch.save(torch.tensor([1]), 'test.pt')

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, embedding_dim, feedforward_dim=2048, dropout_rate = 0.1):
        super().__init__() 
        # We set feedforward_dim as a default to 2048
        self.linear_1 = nn.Linear(embedding_dim, feedforward_dim)
        self.dropout  = nn.Dropout(dropout_rate)
        self.linear_2 = nn.Linear(feedforward_dim, embedding_dim)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear_2(x)
        return x

In [ ]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, h, embedding_dim, dropout_rate = 0.1):
        super().__init__()
        self.norm_1    = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
        self.norm_2    = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
        self.attn      = MultiHeadAttention(h, embedding_dim)
        self.ff        = FeedForward(embedding_dim)
        self.dropout_1 = nn.Dropout(dropout_rate)
        self.dropout_2 = nn.Dropout(dropout_rate)
        
    def forward1(self, x, mask):  
        y = self.norm_1(x) 
        y = self.attn(y, y, y, mask)
        x = x + self.dropout_1(y)
        y = self.norm_2(x)
        y = self.ff(y)
        x = x + self.dropout_2(y)
        
        return x   
    
    def forward(self, x, mask):   
        x = x + self.attn(x, x, x, mask)
        x = self.norm_1(x)   
        x = x + self.ff(x)
        x = self.norm_2(x)
        
        return x   

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size_src, embedding_dim, N, h):
        super().__init__()
        self.N = N
        self.embedder   = Embedder(vocab_size_src, embedding_dim)
        self.posEncoder = PositionalEncoder(embedding_dim) 
        self.layers     = nn.ModuleList([ copy.deepcopy(EncoderLayer(embedding_dim, h)) for i in range(N) ])  
        self.norm       = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
      
    def forward1(self, x, mask):
        x = self.embedder(x)
        x = self.posEncoder(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x) 
    
    def forward(self, x, mask):
        x = self.embedder(x)
        x = self.posEncoder(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return x

In [ ]:
8.  Output Batch Generator
9.  Output Embedding
10. Positional Encoding Tensor Generation 

In [ ]:
vocab_size_fr  = len(vocab_fr) 
output_embedder = Embedder(vocab_size_fr, embedding_dim)

In [ ]:
output_after_embedding = output_embedder(output_fr)

if 1:
    print(output_after_embedding.shape, output_after_embedding.shape, next(output_embedder.parameters()).shape)
    # prove each token has a uniq embedding vector
    # next(output_embedder.parameters()) is a mapping table between output_fr and output_after_embedding
    print(torch.equal(next(output_embedder.parameters())[output_fr[23,4]], output_after_embedding[23,4]) )

In [ ]:
output_after_embedding = output_embedder(output_fr)

if 1:
    print(output_after_embedding.shape, output_after_embedding.shape, next(output_embedder.parameters()).shape)
    # prove each token has a uniq embedding vector
    # next(output_embedder.parameters()) is a mapping table between output_fr and output_after_embedding
    print(torch.equal(next(output_embedder.parameters())[output_fr[23,4]], output_after_embedding[23,4]) )

In [ ]:
11. Masked Multi-Head Attention
12. Residual Connection (Add) and Normalization
13. Multi-Head Attention
14. Residual Connection (Add) and Normalization
15. Feed Forward
16. Residual Connection (Add) and Normalization 
17. Linear Connection
18. Loss Calculation
19. Training

### Masking

In the encoder and decoder: To zero attention outputs wherever there is just padding in the input sentences.
In the decoder: To prevent the decoder ‘peaking’ ahead at the rest of the translated sentence when predicting the next word.

In [ ]:
input_en

In [ ]:
(input_en.T != vocab_en['<PAD>']).unsqueeze(1)

In [22]:








import matplotlib.pyplot as plt

In [ ]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.trg_y = trg[:, 1:]
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [30]:
output_fr

tensor([[  1,   1,   1,   1,   1,   1,   1,   1],
        [  4,  28,  34,  51,  62,  85,  34, 127],
        [  5,  29,  35,  52,  63,  30, 108, 128],
        [  6,  30,  36,  53,  32,  86, 109, 129],
        [  7,  31,  37,  54,  64,  87, 110, 130],
        [  8,  32,  38,  55,  65,  88,  30,  69],
        [  9,  33,  39,  56,  66,  89, 111,  48],
        [ 10,  27,  24,  57,  32,  90,   9, 131],
        [ 11,   2,  40,  58,  67,  69, 112,   9],
        [ 12,   3,  41,  12,  68,  91,  69, 132],
        [ 13,   3,  42,  59,  30,   9, 113, 133],
        [ 14,   3,  43,  60,  69,  92,  32,  57],
        [ 12,   3,  44,  61,  70,  93, 114, 134],
        [ 15,   3,  45,  27,  30,  94, 115, 135],
        [ 16,   3,  46,   2,  71,  95,  32,  69],
        [ 17,   3,  47,   3,  72,  24, 116, 136],
        [ 18,   3,  48,   3,  73,  96, 117, 137],
        [ 10,   3,  49,   3,  32,  44,  22,  27],
        [  7,   3,  50,   3,  74,  97,  92,   2],
        [ 19,   3,  27,   3,  75,  32, 118,   3],


In [23]:

nopeak_mask = lambda size: torch.triu(torch.ones(1, size, size, dtype=int), diagonal=1) == 0

In [25]:
scores = torch.load('scores.pt')
scores.shape

torch.Size([34, 8, 8, 8])

In [28]:
mask = (input_en.T != vocab_fr['<PAD>']).unsqueeze(1) & nopeak_mask(input_en.size(0)) 
mask.shape

torch.Size([8, 34, 34])

In [29]:
mask   = mask.unsqueeze(1)
scores = scores.masked_fill(mask == 0, -1e9)            

RuntimeError: The size of tensor a (34) must match the size of tensor b (8) at non-singleton dimension 3

In [ ]:
output_fr.shape

size = output_fr.T.size(1)
size

plt.figure(figsize=(5,5))
plt.imshow(nopeak_mask(10)[0]) 

(output_fr.T != vocab_fr['<PAD>']).unsqueeze(1).shape

nopeak_mask(output_fr.size(0)).shape

a = (output_fr.T != vocab_fr['<PAD>']).unsqueeze(1)[0]
b = nopeak_mask(output_fr.size(0))[0]

c = torch.tensor([a[0].tolist() for _ in range(38)])

s.shape


for s in a: 
    plt.figure(figsize=(5,5))
    plt.imshow(s) 

output_fr.size(0)

In [ ]:
output_after_embedding = output_after_embedding.transpose(0,1)
#print(output_after_embedding[0])

# creates mask with 0s wherever there is padding in the input
output_mask = (output_fr.T != vocab_fr['<PAD>']).unsqueeze(1)
print(output_mask.shape)
#print(output_mask) 

output_fr.shape

size = output_fr.T.size(1)
size

nopeak_mask = lambda size: torch.triu(torch.ones(1, size, size, dtype=int), diagonal=1) == 0
import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
plt.imshow(nopeak_mask(10)[0]) 

(output_fr.T != vocab_fr['<PAD>']).unsqueeze(1).shape

nopeak_mask(output_fr.size(0)).shape

a = (output_fr.T != vocab_fr['<PAD>']).unsqueeze(1)[0]
b = nopeak_mask(output_fr.size(0))[0]

c = torch.tensor([a[0].tolist() for _ in range(38)])

s.shape

a = (output_fr.T != vocab_fr['<PAD>']).unsqueeze(1) & nopeak_mask(output_fr.size(0)) 
for s in a: 
    plt.figure(figsize=(5,5))
    plt.imshow(s) 

output_fr.size(0)

In [ ]:
# create mask as before
target_seq = output_fr.T 
target_msk = (target_seq != vocab_fr['<PAD>']).unsqueeze(1)
size = target_seq.size(1) # get seq_len for matrix  
target_msk = target_msk & nopeak_mask(size)
target_msk.shape

### Encoder

### Multi-Headed Attention

In [ ]:
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, embedding_dim, h, dropout=0.1):
        super().__init__()
        self.norm_1 = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
        self.norm_2 = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
        self.norm_3 = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(h, embedding_dim)
        self.attn_2 = MultiHeadAttention(h, embedding_dim)
        self.ff     = FeedForward(embedding_dim).cuda()
        
    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x  = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x  = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs, src_mask))
        x2 = self.norm_3(x)
        x  = x + self.dropout_3(self.ff(x2))
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, N, h):
        super().__init__()
        self.N      = N
        self.embed  = Embedder(vocab_size, embedding_dim)
        self.pe     = PositionalEncoder(embedding_dim)
        self.layers = nn.ModuleList([ copy.deepcopy(DecoderLayer(embedding_dim, h)) for i in range(N) ])  
        self.norm   = nn.BatchNorm1d(embedding_dim, eps = 1e-6)
        
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, embedding_dim, N, h):
        super().__init__()
        self.encoder = Encoder(src_vocab, embedding_dim, N, h)
        self.decoder = Decoder(trg_vocab, embedding_dim, N, h)
        self.out     = nn.Linear(embedding_dim, trg_vocab)
        
    def forward(self, src, trg, src_mask, trg_mask):
        e_output = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_output, src_mask, trg_mask)
        output   = self.out(d_output)
        # no softmax on the output bcuz loss function do
        return output 

In [ ]:

model = Transformer(vocab_size_src, vocab_size_trg, embedding_dim, N, h)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
tensor_input, tensor_output = next(iter(train_dl)) 

In [ ]:
tensor_input.T

In [ ]:
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation.
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

def train_model(epochs=0, print_every=100): 
    model.train() 
    
    ts  = time.time()
    total_loss = 0    
    for epoch in range(epochs): 
        for i, (batch_en, batch_fr) in enumerate(train_dl):
            source_input  = batch_en.T
            target_output = batch_fr.T
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            target_input = target_output[:, :-1]
            print(src, batch_trg, trg_input)
            break
            # the words we are trying to predict            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above            
            src_mask, trg_mask = create_masks(src, trg_input)
            
            # Draw DAG
            preds = model(src, trg_input, src_mask, trg_mask)  
            loss = nn.functional.cross_entropy(preds.view(-1, preds.size(-1)), results, ignore_index=target_pad)    
                  
            # SGD
            optim.zero_grad() # we can zero grad anywhere other than between backward() and step()
            loss.backward()
            optim.step()
            
            total_loss += loss.data[0]
            if i % print_every == 0:
                loss_avg = total_loss / print_every
                ts_now = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S') 
                print(f"[{ts_now}][{(time.time()-ts)/60:.1f}m] epoch_{epoch + 1} iter_{i + 1} loss_{loss_avg} ")
                total_loss = 0 
                
        break
                
train_model(epochs=3)

In [ ]:
def translate(model, src, max_len = 80, custom_string=False): 
    model.eval()
    if custom_sentence == True:
        src = tokenize_en(src)
        sentence=Variable(torch.LongTensor([[EN_TEXT.vocab.stoi[tok] for tok in sentence]])).cuda()
        src_mask = (src != input_pad).unsqueeze(-2)
        e_outputs = model.encoder(src, src_mask)
    
    outputs = torch.zeros(max_len).type_as(src.data)
    outputs[0] = torch.LongTensor([FR_TEXT.vocab.stoi['<sos>']])
    
    
    for i in range(1, max_len):    
            
        trg_mask = np.triu(np.ones((1, i, i), k=1).astype('uint8')
        trg_mask= Variable(torch.from_numpy(trg_mask) == 0).cuda()
        
        out = model.out(model.decoder(outputs[:i].unsqueeze(0), e_outputs, src_mask, trg_mask))
        out = nn.functional.softmax(out, dim=-1)
        val, ix = out[:, -1].data.topk(1)
        
        outputs[i] = ix[0][0]
        if ix[0][0] == FR_TEXT.vocab.stoi['<eos>']:
            break
                           
    return ' '.join( [FR_TEXT.vocab.itos[ix] for ix in outputs[:i]] )